# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 30 2023 3:18PM,256229,19,9095101,"Snap Medical Industries, LLC",Released
1,Jan 30 2023 3:18PM,256229,19,9095102,"Snap Medical Industries, LLC",Released
2,Jan 30 2023 3:18PM,256229,19,9095103,"Snap Medical Industries, LLC",Released
3,Jan 30 2023 3:18PM,256229,19,9095131,"Snap Medical Industries, LLC",Released
4,Jan 30 2023 3:09PM,256228,19,N17943,"NAPP Technologies, LLC",Released
5,Jan 30 2023 2:58PM,256227,22,640005,"NBTY Global, Inc.",Released
6,Jan 30 2023 2:51PM,256226,22,640003,"NBTY Global, Inc.",Released
7,Jan 30 2023 2:50PM,256225,22,640000,"NBTY Global, Inc.",Released
8,Jan 30 2023 2:37PM,256223,10,MSP218023,"Methapharm, Inc.",Released
9,Jan 30 2023 2:37PM,256223,10,MSP218024,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,256225,Released,1
33,256226,Released,1
34,256227,Released,1
35,256228,Released,1
36,256229,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256225,NaN,NaN,1.0
256226,NaN,NaN,1.0
256227,NaN,NaN,1.0
256228,NaN,NaN,1.0
256229,NaN,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256145,0.0,15.0,13.0
256148,0.0,0.0,52.0
256149,0.0,0.0,2.0
256152,0.0,0.0,1.0
256154,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256145,0,15,13
256148,0,0,52
256149,0,0,2
256152,0,0,1
256154,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256145,0,15,13
1,256148,0,0,52
2,256149,0,0,2
3,256152,0,0,1
4,256154,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256145,,15,13
1,256148,,,52
2,256149,,,2
3,256152,,,1
4,256154,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 30 2023 3:18PM,256229,19,"Snap Medical Industries, LLC"
4,Jan 30 2023 3:09PM,256228,19,"NAPP Technologies, LLC"
5,Jan 30 2023 2:58PM,256227,22,"NBTY Global, Inc."
6,Jan 30 2023 2:51PM,256226,22,"NBTY Global, Inc."
7,Jan 30 2023 2:50PM,256225,22,"NBTY Global, Inc."
8,Jan 30 2023 2:37PM,256223,10,"Methapharm, Inc."
11,Jan 30 2023 2:36PM,256222,10,Emerginnova
13,Jan 30 2023 2:34PM,256221,10,ISDIN Corporation
17,Jan 30 2023 2:33PM,256220,10,"Methapharm, Inc."
19,Jan 30 2023 2:30PM,256219,19,"NAPP Technologies, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 30 2023 3:18PM,256229,19,"Snap Medical Industries, LLC",,,4
1,Jan 30 2023 3:09PM,256228,19,"NAPP Technologies, LLC",,,1
2,Jan 30 2023 2:58PM,256227,22,"NBTY Global, Inc.",,,1
3,Jan 30 2023 2:51PM,256226,22,"NBTY Global, Inc.",,,1
4,Jan 30 2023 2:50PM,256225,22,"NBTY Global, Inc.",,,1
5,Jan 30 2023 2:37PM,256223,10,"Methapharm, Inc.",,,3
6,Jan 30 2023 2:36PM,256222,10,Emerginnova,,,2
7,Jan 30 2023 2:34PM,256221,10,ISDIN Corporation,,,4
8,Jan 30 2023 2:33PM,256220,10,"Methapharm, Inc.",,,2
9,Jan 30 2023 2:30PM,256219,19,"NAPP Technologies, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 30 2023 3:18PM,256229,19,"Snap Medical Industries, LLC",4,,
1,Jan 30 2023 3:09PM,256228,19,"NAPP Technologies, LLC",1,,
2,Jan 30 2023 2:58PM,256227,22,"NBTY Global, Inc.",1,,
3,Jan 30 2023 2:51PM,256226,22,"NBTY Global, Inc.",1,,
4,Jan 30 2023 2:50PM,256225,22,"NBTY Global, Inc.",1,,
5,Jan 30 2023 2:37PM,256223,10,"Methapharm, Inc.",3,,
6,Jan 30 2023 2:36PM,256222,10,Emerginnova,2,,
7,Jan 30 2023 2:34PM,256221,10,ISDIN Corporation,4,,
8,Jan 30 2023 2:33PM,256220,10,"Methapharm, Inc.",2,,
9,Jan 30 2023 2:30PM,256219,19,"NAPP Technologies, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 30 2023 3:18PM,256229,19,"Snap Medical Industries, LLC",4,,
1,Jan 30 2023 3:09PM,256228,19,"NAPP Technologies, LLC",1,,
2,Jan 30 2023 2:58PM,256227,22,"NBTY Global, Inc.",1,,
3,Jan 30 2023 2:51PM,256226,22,"NBTY Global, Inc.",1,,
4,Jan 30 2023 2:50PM,256225,22,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 30 2023 3:18PM,256229,19,"Snap Medical Industries, LLC",4,NaN,NaN
1,Jan 30 2023 3:09PM,256228,19,"NAPP Technologies, LLC",1,NaN,NaN
2,Jan 30 2023 2:58PM,256227,22,"NBTY Global, Inc.",1,NaN,NaN
3,Jan 30 2023 2:51PM,256226,22,"NBTY Global, Inc.",1,NaN,NaN
4,Jan 30 2023 2:50PM,256225,22,"NBTY Global, Inc.",1,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 30 2023 3:18PM,256229,19,"Snap Medical Industries, LLC",4,0.0,0.0
1,Jan 30 2023 3:09PM,256228,19,"NAPP Technologies, LLC",1,0.0,0.0
2,Jan 30 2023 2:58PM,256227,22,"NBTY Global, Inc.",1,0.0,0.0
3,Jan 30 2023 2:51PM,256226,22,"NBTY Global, Inc.",1,0.0,0.0
4,Jan 30 2023 2:50PM,256225,22,"NBTY Global, Inc.",1,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4099080,188,19.0,0.0
12,512374,2,0.0,0.0
15,256212,2,17.0,0.0
16,512306,2,0.0,0.0
19,1024862,20,21.0,20.0
21,512390,2,0.0,0.0
22,768678,3,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4099080,188,19.0,0.0
1,12,512374,2,0.0,0.0
2,15,256212,2,17.0,0.0
3,16,512306,2,0.0,0.0
4,19,1024862,20,21.0,20.0
5,21,512390,2,0.0,0.0
6,22,768678,3,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,188,19.0,0.0
1,12,2,0.0,0.0
2,15,2,17.0,0.0
3,16,2,0.0,0.0
4,19,20,21.0,20.0
5,21,2,0.0,0.0
6,22,3,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,188.0
1,12,Released,2.0
2,15,Released,2.0
3,16,Released,2.0
4,19,Released,20.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21,22
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,20.0,0.0,0.0
Executing,19.0,0.0,17.0,0.0,21.0,0.0,0.0
Released,188.0,2.0,2.0,2.0,20.0,2.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21,22
0,Completed,0.0,0.0,0.0,0.0,20.0,0.0,0.0
1,Executing,19.0,0.0,17.0,0.0,21.0,0.0,0.0
2,Released,188.0,2.0,2.0,2.0,20.0,2.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21,22
0,Completed,0.0,0.0,0.0,0.0,20.0,0.0,0.0
1,Executing,19.0,0.0,17.0,0.0,21.0,0.0,0.0
2,Released,188.0,2.0,2.0,2.0,20.0,2.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()